In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.getOrCreate()

val data = (spark.read.option("header", "true")
            .option("inferSchema", "true")
            .format("csv")
            .option("sep", ";")
            .load("C:/Users/ibhat/Desktop/Projects/Bank_Marketing/bank.csv"))

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.137.1:4040
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1501538400462)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6e331193
data: org.apache.spark.sql.DataFrame = [age: int, job: string ... 15 more fields]


In [2]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [3]:
data.columns

res1: Array[String] = Array(age, job, marital, education, default, balance, housing, loan, contact, day, month, duration, campaign, pdays, previous, poutcome, y)


In [4]:
data.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

##### All feature 
age, job, marital, education, default, balance, housing, loan, contact, day, month, duration, campaign, pdays, previous, poutcome

##### Features being Considered
age, job, marital,  default, balance, housing, loan, duration, campaign, pdays, previous, poutcome

In [5]:
data.groupBy("poutcome").count().show()

+--------+-----+
|poutcome|count|
+--------+-----+
| success| 1511|
| unknown|36959|
|   other| 1840|
| failure| 4901|
+--------+-----+



'poutcome' seems to be important feature. However, the previous outcome was unknown for majority of the customers. So, It is better to drop this feature.

##### Final Features being Considered
age, job, marital,  default, balance, housing, loan, duration, campaign, pdays

In [6]:
val final_data = data.select("age", "job", "marital", "default", "balance", "housing", "loan", "duration", "campaign", "pdays", "y")
final_data.describe().show()

+-------+------------------+-------+--------+-------+------------------+-------+-----+-----------------+-----------------+------------------+-----+
|summary|               age|    job| marital|default|           balance|housing| loan|         duration|         campaign|             pdays|    y|
+-------+------------------+-------+--------+-------+------------------+-------+-----+-----------------+-----------------+------------------+-----+
|  count|             45211|  45211|   45211|  45211|             45211|  45211|45211|            45211|            45211|             45211|45211|
|   mean| 40.93621021432837|   null|    null|   null|1362.2720576850766|   null| null|258.1630797814691|2.763840658246887| 40.19782796222158| null|
| stddev|10.618762040975405|   null|    null|   null|3044.7658291685257|   null| null|257.5278122651706|3.098020883279192|100.12874599059828| null|
|    min|                18|  admin|divorced|     no|             -8019|     no|   no|                0|        

final_data: org.apache.spark.sql.DataFrame = [age: int, job: string ... 9 more fields]


There is no null values in the columns being considered. Good to go

In [8]:
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoder}


// Converting string values to numerical column
val JobIndexer = new StringIndexer().setInputCol("job").setOutputCol("JobIndex")
val MaritalIndexer = new StringIndexer().setInputCol("marital").setOutputCol("MaritalIndex")
val DefaultIndexer = new StringIndexer().setInputCol("default").setOutputCol("DefaultIndex")
val HousingIndexer = new StringIndexer().setInputCol("housing").setOutputCol("HousingIndex")
val LoanIndexer = new StringIndexer().setInputCol("loan").setOutputCol("LoanIndex")
val OutcomeIndexer = new StringIndexer().setInputCol("y").setOutputCol("label")

// Using OneHotEncoder to avoid hierarchy in numerical value obtaied in above step
val JobEncoder = new OneHotEncoder().setInputCol("JobIndex").setOutputCol("JobVec")
val MaritalEncoder = new OneHotEncoder().setInputCol("MaritalIndex").setOutputCol("MaritalkVec")

// All the other columns have binary values(either Yes or No). So no need to hot-encode

import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoder}
JobIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_48eff509bb4a
MaritalIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_7eb1612fb40b
DefaultIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_fe91b1088f88
HousingIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_a63c3c892fbf
LoanIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_6d0bab2d72a9
OutcomeIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_85b75a1400c5
JobEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_0a2ad65186d7
MaritalEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_3541bf8bfe6a


In [9]:
// Assemble everything together to be ("label","features") format
val assembler = (new VectorAssembler()
                .setInputCols(Array("age", "JobVec", "MaritalkVec", "DefaultIndex", "balance", 
                                    "HousingIndex", "LoanIndex", "duration", "campaign", "pdays"))
                .setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_4c0cab1c7ca3


In [15]:
// Scaling the features
import org.apache.spark.ml.feature.StandardScaler

val scaler = (new StandardScaler()
              .setInputCol("features")
              .setOutputCol("scaledFeatures"))

import org.apache.spark.ml.feature.StandardScaler
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_7718459a2d97


In [16]:
// Spliting the data into Training set and Test set
val Array(training, test) = final_data.randomSplit(Array(0.7, 0.3), seed = 12345)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [age: int, job: string ... 9 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [age: int, job: string ... 9 more fields]


In [20]:
// Defining Classifier Model
import org.apache.spark.ml.classification.{LogisticRegression, RandomForestClassifier}

val lr = new LogisticRegression()
val rcf = new RandomForestClassifier().setMaxDepth(10).setNumTrees(300)

import org.apache.spark.ml.classification.{LogisticRegression, RandomForestClassifier}
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_bd73315b1264
rcf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_586ace129f3e


In [21]:
// Set Up the Pipeline
import org.apache.spark.ml.Pipeline
// This creates the stages for the task we were trying to accomplish in last few stages before

// *************************** For Logistic Regressor ***************************
// val pipeline = new Pipeline().setStages(Array(JobIndexer, MaritalIndexer, DefaultIndexer, HousingIndexer,
//                                               LoanIndexer, OutcomeIndexer, JobEncoder, MaritalEncoder, assembler, scaler, lr))

// *************************** For Random Forest Classifier ***************************
val pipeline = new Pipeline().setStages(Array(JobIndexer, MaritalIndexer, DefaultIndexer, HousingIndexer,
                                              LoanIndexer, OutcomeIndexer, JobEncoder, MaritalEncoder, assembler, scaler, rcf))

import org.apache.spark.ml.Pipeline
pipeline: org.apache.spark.ml.Pipeline = pipeline_cb6478b8b9f6


In [22]:
// Fitting the model
val model = pipeline.fit(training)

// Geting results on Test set
val results = model.transform(test)

model: org.apache.spark.ml.PipelineModel = pipeline_cb6478b8b9f6
results: org.apache.spark.sql.DataFrame = [age: int, job: string ... 22 more fields]


In [23]:
results.select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [24]:
// Model Evaluation

// Model selection is still in RDD phase. So We will use rdd here instead of spark dataframe. 
// This will be update in future versions of Spark
import org.apache.spark.mllib.evaluation.MulticlassMetrics

val predictionAndLabels = results.select($"prediction",$"label").as[(Double, Double)].rdd


import org.apache.spark.mllib.evaluation.MulticlassMetrics
predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[351] at rdd at <console>:64


In [25]:
val metrics = new MulticlassMetrics(predictionAndLabels)

// Confusion Matrix
println("Confusion Matrix")
println(metrics.confusionMatrix)
println(s"Accuracy: ${metrics.accuracy}")
println(s"Precision : ${metrics.weightedPrecision}")

Confusion Matrix
11932.0  191.0  
1258.0   376.0  
Accuracy: 0.8946718034455187
Precision : 0.875942072636317


metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@105bbc9f
